In [1]:
import os
import pandas as pd
if os.name == 'nt':
    os.environ["OMP_NUM_THREADS"] = '2'
import warnings
warnings.filterwarnings('ignore')
import sys
sys.path.append('../')
from utiliT.io import read_pickle, dump_pickle
from scripts.instance import *
from scripts.benders import *
import gc
from tqdm.notebook import tqdm

In [2]:
def runner(function, files = None):
    bad_instances = []
    if files is None:
        files = os.listdir("../data")
    for file in tqdm(files):
        instance_name = file[:-4]
        instance = read_pickle(f"../data/{file}")
        try:
            result = function(instance)
            result['instance_name'] = instance_name
            file_name = f"../results/{result['instance_name']}_{result['method']}.pkl"
            dump_pickle(result, file_name)
            del result
        except:
            bad_instances.append(file)
            print(file)
        del result
        gc.collect()

In [3]:
def multi_runner(functions, files = None):
    print(f"Running {len(functions)} functions")
    i = 1
    for function in functions:
        print(f"Running function: {i}")
        i = i + 1
        runner(function, files=files)
        gc.collect()

In [4]:
def clean_results(terms):
    for term in terms:
        files = os.listdir("../results")
        deleting = [x for x in files if term in x]
        for f in deleting:
            os.remove(f"../results/{f}")

In [5]:
clean_results(["dropout", "hybrid-cut"])

In [ ]:
functions = [
    lambda x: dropout_cut(x, "kmeans"),
    lambda x: dropout_cut(x, "affinity"),
    lambda x: dropout_cut(x, "hierarchical"),
    lambda x: dropout_cut(x, "spectral"),
    lambda x: dropout_cut(x, "random"),
    lambda x: hybrid(x, "kmeans"),
    lambda x: hybrid(x, "affinity"),
    lambda x: hybrid(x, "hierarchical"),
    lambda x: hybrid(x, "spectral"),
    lambda x: hybrid(x, "random"),
]
multi_runner(functions)

TONIGHT: RUN EVERYTHING BELOW THIS